In [18]:
import requests
from bs4 import BeautifulSoup
import re
import math
import itertools
import json
import urllib
from pandas.io.json import json_normalize
import pandas as pd

# Main URL - We have to get accessvariable hash code to proof we are human
main_url = 'https://resistancemap.cddep.org/AntibioticResistance.php'

# Data URL - Send parameteres with hash code
data_url = 'https://resistancemap.cddep.org/getData.php'

# Pathogen name and IDs - Defined in Parameters Dictionary
pathogens = [['Acinetobacter baumannii', 8],
             ['Enterobacter aerogenes cloacae', 9],
             ['Enterococcus faecalis', 1], ['Enterococcus faecium', 2],
             ['Escherichia coli', 3], ['Pseudomonas aeruginosa', 7],
             ['Klebsiella pneumoniae', 4], ['Salmonella Paratyphi', 10],
             ['Salmonella Typhi', 11], ['Staphylococcus aureus', 5],
             ['Streptococcus pneumoniae', 6]]

# Add all IDs to get whole data
parameters = {
    'name': pathogens[10][0],  # PATHOGEN Name
    'orgId': pathogens[10][1],  # PATHOGEN ID
    'abxId[]': [
        12, 4, 49, 2, 15, 48, 5, 13, 6, 7, 18, 17, 10, 8, 11, 14, 19, 9, 39,
        16, 3
    ],  # All ANTIBIOTICS
    'countryId[]': [
        19, 37, 38, 17, 33, 44, 43, 35, 70, 1, 78, 92, 82, 26, 87, 93, 36, 39,
        2, 40, 3, 98, 4, 6, 8, 7, 9, 10, 11, 12, 106, 13, 14, 15, 16, 18, 19,
        102, 21, 22, 41, 23, 110, 25, 27, 28, 29, 30, 34, 32, 31, 42, 89, 65,
        69, 73, 83, 5, 95, 94, 90, 112, 24, 74, 105, 81, 97, 109, 108, 111,
        104, 20, 101, 100, 85
    ],  # All Countries
    'charttype': 'chart',
    'page': 'resistance', 
    'resetabxs': 'false',
    'defaultValues': 'false',
}


def get_source(url):
    source = requests.get(url)
    soup = BeautifulSoup(source.content, 'html.parser')
    return soup


def get_access_variable(url):
    soup = BeautifulSoup(requests.get(url).content, 'html.parser')
    scripts = soup.findAll('script')
    input = soup.findAll('script', type="text/javascript")[2]
    accessvariable = re.search(r'accessvariable: "(.*)"', str(input))[1]
    return str(accessvariable)


def save_xlsx(data, excel_name):
    writer = pd.ExcelWriter(excel_name + '.xlsx', engine='xlsxwriter')
    df.to_excel(writer, sheet_name='Sheet1', index=False, startrow=1, header=False)
    worksheet = writer.sheets['Sheet1']
    workbook = writer.book
    header_format = workbook.add_format({
        'bold': True,
        'valign': 'top',
        #'fg_color': '#ffcccc',
        'border': 0
    })
    worksheet.set_row(0, None, header_format)
    for col_num, value in enumerate(data.columns.values):
        worksheet.write(0, col_num, value, header_format)
    writer.save()

# Hash Code
parameters["accessvariable"] = get_access_variable(main_url)

# Request and get Json
r = requests.post(data_url, data=parameters).json()

#json.dumps(r)

# Append to dataframe
df = pd.DataFrame(r[3])

# Save Dataframe to excel
save_xlsx(df, parameters['name'])